In [4]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, RNN, Lambda, Concatenate
from keras.layers.merge import Multiply
from keras.models import Model
from keras.losses import mean_squared_error, binary_crossentropy
from keras.models import Model
import keras.backend as K
from sklearn.metrics import mean_squared_error as mse

Using TensorFlow backend.


# Right now make as many assumptions as possible

It's better to ask it
- derive speeds
- separate x and y. mby use same module for both?
- learn bounces separately
     - is LSTM much better if there is only 1 frame after the bounce vs 0?

In [5]:
def save_model(model, model_str):
    model_json = model.to_json()
    with open('trained/'+model_str+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights('trained/'+model_str+".h5")
    print("Saved model to disk")

In [6]:
## Straight movement, derive speed

from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, RNN, Lambda

# This returns a tensor
inputs = Input(shape=(4,1,))
# inputs = Input(shape=(10,1))

# a layer instance is callable on a tensor, and returns a tensor
x = LSTM(4, return_sequences=False)(inputs)
x = Dense(2)(x)

predictions = x#Dense(2, input_shape=(4,1))(x)

# This creates a model that includes
# the Input layer and three Dense layers
model_ws = Model(inputs=inputs, outputs=predictions)
model_ws.compile(loss=mean_squared_error,
              optimizer='adam',
              metrics=['accuracy'])
model_ws.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 1)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4)                 96        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 106
Trainable params: 106
Non-trainable params: 0
_________________________________________________________________


In [7]:
df = pd.read_csv('data/sim_data2.csv')
# x, x_vel, y, y_vel
a = df.to_numpy()
a = a.reshape((20000,5,4)) #samples x timesteps x features
X = a[:,:4,0].reshape(20000,4,1) #np.delete(a, [1,3], axis=2)[:,:10,:]
Y = a[:,4,:2] #.reshape(10000,5,2)

train_X = X[:8000]
train_Y = Y[:8000]

# test_X = X[8000:]
# test_Y = Y[8000:]
model_ws.fit(train_X, train_Y, epochs=200, verbose=False) #200 seems reasonable

InternalError: Blas GEMM launch failed : a.shape=(32, 4), b.shape=(4, 4), m=32, n=4, k=4
	 [[{{node lstm_1/while/MatMul_7}} = MatMul[T=DT_FLOAT, _class=["loc:@training/Adam/gradients/AddN_2"], transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](lstm_1/while/Switch_3:1, lstm_1/while/MatMul_7/Enter)]]
	 [[{{node metrics/acc/Mean/_83}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1691_metrics/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(model_ws.history.history['loss'][40:])

In [ ]:
test_X = X[8000:]
test_Y = Y[8000:]
mse(test_Y, model_ws.predict(test_X))**0.5

In [ ]:
mse(train_Y, model_ws.predict(train_X))**0.5

In [ ]:
model_ws.predict(train_X[[100]])

In [ ]:
save_model(model_ws, 'model_ws')

# Bounces

In [ ]:
inputs = Input(shape=(2,))

x = Dense(2)(inputs)
x = Dense(4, activation='relu')(x)
x = Dense(2)(x)
predictions = x#Dense(4, input_shape=(4,1))(x)

# This creates a model that includes
# the Input layer and three Dense layers
model_b = Model(inputs=inputs, outputs=predictions)
model_b.compile(loss=mean_squared_error,
              optimizer='adam',
              metrics=['accuracy'])
model_b.summary()

In [ ]:
df = pd.read_csv('data/bounces_mirror.csv')
a = df.to_numpy()
a = a.reshape((20000,2,4)) #samples x timesteps x features
a = a[:,:,:2]
train_X = a[:18000,0,:]
test_X = a[18000:,0,:]
train_Y = a[:18000,1,:]
test_Y = a[18000:,1,:]
model_b.fit(train_X, train_Y, epochs=600, verbose=False) #e = 150 is enough

In [ ]:
save_model(model_b, 'model_b')

In [ ]:
print(mse(test_Y, model_b.predict(test_X))**0.5)
print(mse(train_Y, model_b.predict(train_X))**0.5)

In [ ]:
plt.plot(model_b.history.history['loss'])

In [ ]:
train_X[:5]

In [ ]:
model_b.predict(train_X[:5])

In [ ]:
train_Y[[0,1]]

In [ ]:
save_model(model_b, 'model_b')

### Attention

In [ ]:
inputs = Input(shape=(1,))
x = Dense(8)(inputs)
x = Dense(20, activation='relu')(x)
x = Dense(2, activation='softmax')(x)
predictions = x

model_att = Model(inputs=inputs, outputs=predictions)
model_att.compile(loss=binary_crossentropy,
              optimizer='adamax',
              metrics=['accuracy'])
model_att.summary()

In [ ]:
df = pd.read_csv('data/bounces_att.csv')
a = df.to_numpy()
train_X = a[:,0].reshape(20000,1)
train_Y = (a[:,2] == True)

l = train_X.shape[0]
i = np.arange(l)
t = np.zeros((l, 2))
t[:,0] = 1-train_Y
t[:,1] = train_Y
train_Y = t

model_att.fit(train_X, train_Y, epochs=100, verbose=False)

In [ ]:
save_model(model_att, 'model_att')

In [ ]:
plt.plot(model_att.history.history['acc'])

In [ ]:
inputs = Input(shape=(1,))

x = model_att(inputs)
x = Lambda(lambda s: K.round(s))(x)
predictions = x
model_lambda = Model(inputs=inputs, outputs=predictions)
model_lambda.compile(loss=binary_crossentropy,
              optimizer='adam',
              metrics=['acc'])
model_lambda.summary()

In [ ]:
save_model(model_lambda, 'model_lambda')

## Bounce series

In [ ]:
df = pd.read_csv('data/bounces_x_1.csv')
a = df.to_numpy()
a = a.reshape((1000,5,4))

X = a[:,:4,0].reshape(1000,4,1) #np.delete(a, [1,3], axis=2)[:,:10,:]
Y = a[:,4,:2] #.reshape(10000,5,2)

train_X = X[:800]
train_Y = Y[:800]

test_X = X[800:]
test_Y = Y[800:]

In [ ]:
isBounce = K.tf.convert_to_tensor([0,1],dtype=np.float32) #if attention outputs this, then bounce module is needed

inputs_rec = Input(shape=(4,1), name='input_rec')
rec = model_ws(inputs_rec)
rec_x = Lambda(lambda s: K.reshape(s[:,0], (-1,1)))(rec)
bnc = model_b(rec)
att = model_lambda(rec_x)
att2 = Lambda(lambda s: K.switch(K.equal(s[0], isBounce), s[1], s[2]))([att, bnc, rec]) #~hard attention

x = att2

model_m = Model(inputs=inputs_rec, outputs=x)
model_m.compile(loss=mean_squared_error,
              optimizer='adam',
              metrics=['accuracy'])
model_m.summary()

In [ ]:
save_model(model_m, 'model_m')

In [ ]:
df2 = pd.read_csv('data/sim_data_borders.csv')
# x, x_vel, y, y_vel
a2 = df2.to_numpy()
a2 = a2.reshape((4000,5,4)) #samples x timesteps x features
X2 = a2[:,:4,0].reshape(4000,4,1) #np.delete(a, [1,3], axis=2)[:,:10,:]
Y2 = a2[:,4,:2] #.reshape(10000,5,2)

train_X2 = X2[:8000]
train_Y2 = Y2[:8000]

In [ ]:
mixdf = pd.concat([df, df2[:5000]])
mix = mixdf.to_numpy()
mix = mix.reshape(2000,5,4)
np.random.shuffle(mix)
mix_train_x = mix[:1800,:4,0].reshape(1800,4,1)
mix_train_y = mix[:1800,4,:2]
mix_test_x = mix[1800:,:4,0].reshape(200,4,1)
mix_test_y = mix[1800:,4,:2]
mse(mix_test_y, model_m.predict(mix_test_x)) # My model